## Clip-T & Clip-I

In [ ]:
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
import torch
import os
import math
import re
from itertools import groupby
from PIL import Image

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append('../')

from lora_diffusion import evaluate_pipe, prepare_clip_model_sets, LoraInjectedConv2d, LoraInjectedLinear, patch_pipe, tune_lora_scale, parse_safeloras
from lora_diffusion.lora import _find_modules, UNET_CROSSATTN_TARGET_REPLACE, DEFAULT_TARGET_REPLACE
from visualization.vis_image import visualize_images
import safetensors
import time

os.environ["DISABLE_TELEMETRY"] = 'YES'
os.environ["HTTP_PROXY"] = "http://localhost:8890"
os.environ["HTTPS_PROXY"] = "http://localhost:8890"
model_id = "runwayml/stable-diffusion-v1-5"
device = torch.device("cuda:6")
time.sleep(0.01)
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16,local_ckpt_files_only=True,revision='39593d5650112b4cc580433f6b0435385882d819').to(device)
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
torch.manual_seed(0)

import glob
from collections import defaultdict
preped_clip = prepare_clip_model_sets()
instance_data_dir = '/data/zhicai/code/Text-regularized-customization/custom_datasets/data/dog'
class_tokens = 'dog'
custom_token = '<krk1> dog'

lora_ckpts = sorted(glob.glob("../outputs/checkpoints/dog/ewc_reg_shareTI/baseline_lora_rank_10/lora_weight_s*.safetensors"))
logs = defaultdict(list)
for lora_ckpt in lora_ckpts:
    print(lora_ckpt)
    patch_pipe(
        pipe,
        lora_ckpt,
        patch_text=False,
        patch_ti=True,
        patch_unet=True,
        filter_crossattn_str = 'cross+self'
    )


    test_image_path = instance_data_dir
    images = []
    for file in os.listdir(test_image_path):
        if (
            file.lower().endswith(".png")
            or file.lower().endswith(".jpg")
            or file.lower().endswith(".jpeg")
        ):
            images.append(
                Image.open(os.path.join(test_image_path, file))
            )

    log = evaluate_pipe(
        pipe,
        target_images=images,
        class_token=class_tokens,
        learnt_token=custom_token,
        n_test=10,
        n_step=50,
        clip_model_sets=preped_clip,
    ),
    logs['text_alignment_avg'].append(log["text_alignment_avg"])
    logs['image_alignment_avg'].append(log["image_alignment_avg"])


In [ ]:
import glob
from collections import defaultdict
instance_data_dir = '/data/zhicai/code/Text-regularized-customization/custom_datasets/data/dog'
class_tokens = 'dog'
custom_token = '<krk1> dog'

lora_ckpts = sorted(glob.glob("../outputs/checkpoints/dog/ewc_reg_shareTI/ewc_gen_weight_0.01_rank_10_seed_0/lora_weight_s*.safetensors"))
logs = defaultdict(list)
for lora_ckpt in lora_ckpts:
    print(lora_ckpt)
    patch_pipe(
        pipe,
        lora_ckpt,
        patch_text=False,
        patch_ti=True,
        patch_unet=True,
        filter_crossattn_str = 'cross+self'
    )


    test_image_path = instance_data_dir
    images = []
    for file in os.listdir(test_image_path):
        if (
            file.lower().endswith(".png")
            or file.lower().endswith(".jpg")
            or file.lower().endswith(".jpeg")
        ):
            images.append(
                Image.open(os.path.join(test_image_path, file))
            )

    log = evaluate_pipe(
        pipe,
        target_images=images,
        class_token=class_tokens,
        learnt_token=custom_token,
        n_test=10,
        n_step=50,
        clip_model_sets=preped_clip,
    ),
    logs['text_alignment_avg'].append(log[0]["text_alignment_avg"])
    logs['image_alignment_avg'].append(log[0]["image_alignment_avg"])
    print(log[0]["text_alignment_avg"])


## class KID
先通过base/mp_sample_lora.py和base/mp_sample.py分别采样得到 tune 过之后的 <class> 图片和预训练模型的<class> 图片，然后再计算两者 kid。

In [ ]:
import subprocess
import sys
import os
sys.path.append('/data/zhicai/code/Text-regularized-customization/')
from base.utils import parse_templates_class_name, DEBUG_DIRS
sys.path.append(os.getcwd())

experiment_types = ['shareTI_text_0.01_norm_0.01', 
                    'shareTI_text_0.01_G-ewc_0.01',
                    'shareTI_text_0.01_L-ewc_0.01',
                    'shareTI_text_0.01_L-priorloss']
    # 要运行的Shell命令
for data_dir_name in DEBUG_DIRS:
    
    _, class_name = parse_templates_class_name(data_dir_name)

    dir1_template = '/data/zhicai/code/Text-regularized-customization/custom_datasets/sample/{}'
    dir2_template ='/data/zhicai/code/Text-regularized-customization/outputs/debug/sample/{}/{}/decay_lr_5e-4_gas_4/lora_weight_s2000/class'
                
    for experiment_type in experiment_types:
        dir2 = dir2_template.format(data_dir_name, experiment_type)
        dir1 = dir1_template.format(class_name)
        shell_command = f"python -m pytorch_fid {dir1} {dir2} --device cuda:2"

        # 使用subprocess运行Shell命令
        result = subprocess.run(shell_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        # 打印命令的标准输出
        print(f"Command Output for {data_dir_name} and {experiment_type}:")
        print(result.stdout)

    # # 打印命令的标准错误输出
    # print("Command Error:")
    # print(result.stderr)

    # 打印命令的返回值
    # print("Exit Code:", result.returncode)


In [ ]:
import subprocess
import sys
import os
sys.path.append('/data/zhicai/code/Text-regularized-customization/')
from base.utils import parse_templates_class_name, DEBUG_DIRS
sys.path.append(os.getcwd())

experiment_types = ['shareTI_text_0.01_norm_0.01', 'shareTI_text_0.01_G-ewc_0.01','shareTI_text_0.01_L-ewc_0.01','shareTI_text_0.01_L-ewc_priorloss']
    # 要运行的Shell命令
for data_dir_name in DEBUG_DIRS:
    _, class_name = parse_templates_class_name(data_dir_name)

    dir2_template ='/data/zhicai/code/Text-regularized-customization/outputs/debug/sample/{}/{}/decay_lr_5e-4_gas_4/lora_weight_s2000/class'
                
    dir1_template = '/data/zhicai/code/Text-regularized-customization/custom_datasets/sample/{}'
    for experiment_type in experiment_types:
        dir2 = dir2_template.format(data_dir_name,experiment_type)    
        dir1 = dir1_template.format(class_name)
        shell_command = f"python -m pytorch_fid {dir1} {dir2} --device cuda:2"

        # 使用subprocess运行Shell命令
        result = subprocess.run(shell_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        # 打印命令的标准输出
        print(f"Command Output for {data_dir_name} and {experiment_type}:")
        print(result.stdout)

    # # 打印命令的标准错误输出
    # print("Command Error:")
    # print(result.stderr)

    # 打印命令的返回值
    # print("Exit Code:", result.returncode)


In [ ]:
import subprocess
import sys
import os
sys.path.append('/data/zhicai/code/Text-regularized-customization/')
from base.utils import parse_templates_class_name
sys.path.append(os.getcwd())

# 要运行的Shell命令
data_dir_name = 'robot_toy'
_, class_name = parse_templates_class_name(data_dir_name)

dir2_list =['/data/zhicai/code/Text-regularized-customization/outputs/sample/{}/shareTI_text_0.01/decay_lr_5e-4_gas_4/lora_weight_s2000',]
dir1 = '/data/zhicai/code/Text-regularized-customization/custom_datasets/sample/{}'
for dir2 in dir2_list:
    dir2 = dir2.format(data_dir_name)    
    dir1 = dir1.format(class_name)
    shell_command = f"python -m pytorch_fid {dir1} {dir2} --device cuda:2"

    # 使用subprocess运行Shell命令
    result = subprocess.run(shell_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # 打印命令的标准输出
    print("Command Output:")
    print(result.stdout)

    # # 打印命令的标准错误输出
    # print("Command Error:")
    # print(result.stderr)

    # 打印命令的返回值
    # print("Exit Code:", result.returncode)



